In [9]:

# TODOS
# need to extract weights at each training step
# resume from checkpoint thing
# get traceback form indices to raw training data
# Get back form 64x64 to 224x224 sized training images
# Add some other super lame shit features andho
import os
import sys
sys.path.append("/home/zsarwar/data_sharing/decile/cords")

os.environ['CUDA_VISIBLE_DEVICES']='1'
os.chdir("/home/zsarwar/data_sharing/decile/cords")
import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from cords.utils.data.datasets.SL import gen_dataset
from torch.utils.data import Subset
from cords.utils.config_utils import load_config_data
import os.path as osp
from cords.utils.data.data_utils import WeightedSubset
from ray import tune
from gen_data import get_datasets
from tqdm import tqdm
from torchvision.models import ResNet50_Weights, resnet18, resnet50, ResNet18_Weights
from cords.utils.models import ResNet50, ResNet18
import pickle
import pandas as pd


trn_batch_size = 16
val_batch_size = 512
tst_batch_size = 1000
numclasses = 7
num_epochs = 200
print_every = 10
save_every = 20
data_fraction = 0.1
select_subset_every = 2
pretrained = False
arch = 'ResNet18'

root_save_dir = "/bigstor/zsarwar/GradMatch"
config = f'DvW_Imagenet_testing_num_epochs-{num_epochs}_pretrained-{pretrained}_model-{arch}_numclasses-{numclasses}_fraction-{data_fraction}_batchsize-{trn_batch_size}_Perbatch_DvW_Pretrained'

out_dir = os.path.join(root_save_dir, config)
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

# Make ckpt directory
ckpt_dir = os.path.join(out_dir, 'checkpoints')
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

stats_path = os.path.join(out_dir, 'stats.pkl')

#trainset, validset, testset, num_cls = gen_dataset('/home/zsarwar/data_sharing/decile/cords/data/', 'cifar10', None, isnumpy=False)

trainset, validset, testset = get_datasets()




#trn_batch_size = 128
#val_batch_size = 512
#tst_batch_size = 1000

# Creating the Data Loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=trn_batch_size,
                                          shuffle=False, pin_memory=True)

valloader = torch.utils.data.DataLoader(validset, batch_size=val_batch_size,
                                        shuffle=False, pin_memory=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=tst_batch_size,
                                          shuffle=False, pin_memory=True)




#numclasses = 7
device = 'cuda' #Device Argument

#model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
# Change classification layer
#model.fc = nn.Linear(in_features=512, out_features=numclasses)
#model.embDim = 512
#model = ResNet18()
model = ResNet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(in_features=512, out_features=numclasses)

# Add code to freeze

#model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1), bias=False)
#model.maxpool = nn.Identity()
model = model.to(device)



In [10]:

for param in model.named_parameters():
    print(param[1][0])
    break

tensor([[[-1.0419e-02, -6.1356e-03, -1.8098e-03,  7.4841e-02,  5.6615e-02,
           1.7083e-02, -1.2694e-02],
         [ 1.1083e-02,  9.5276e-03, -1.0993e-01, -2.8050e-01, -2.7124e-01,
          -1.2907e-01,  3.7424e-03],
         [-6.9434e-03,  5.9089e-02,  2.9548e-01,  5.8720e-01,  5.1972e-01,
           2.5632e-01,  6.3573e-02],
         [ 3.0505e-02, -6.7018e-02, -2.9841e-01, -4.3868e-01, -2.7085e-01,
          -6.1282e-04,  5.7602e-02],
         [-2.7535e-02,  1.6045e-02,  7.2595e-02, -5.4102e-02, -3.3285e-01,
          -4.2058e-01, -2.5781e-01],
         [ 3.0613e-02,  4.0960e-02,  6.2850e-02,  2.3897e-01,  4.1384e-01,
           3.9359e-01,  1.6606e-01],
         [-1.3736e-02, -3.6746e-03, -2.4084e-02, -6.5877e-02, -1.5070e-01,
          -8.2230e-02, -5.7828e-03]],

        [[-1.1397e-02, -2.6619e-02, -3.4641e-02,  3.6812e-02,  3.2521e-02,
           6.6221e-04, -2.5743e-02],
         [ 4.5687e-02,  3.3603e-02, -1.0453e-01, -3.0885e-01, -3.1253e-01,
          -1.6051e-01, -1.2

In [4]:
weights = ResNet18_Weights.IMAGENET1K_V1

In [7]:
weights.get_state_dict(progress = True, check_hash=True)

TypeError: get_state_dict() got an unexpected keyword argument 'check_hash'